<a href="https://colab.research.google.com/github/nour-ezzehi/Travail-des-Femmes-Agricultrices/blob/main/DataAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing necessary libraries

In [406]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

Load the Dataset

In [407]:
df = pd.read_csv("sample_data/Cleaned SST en agriculture.csv")
df.head()
df.columns

Index(['N°', 'Date de la visite', 'Nom', 'Prénom', 'N° du téléphone',
       'Date de naissance', 'Age', 'Situation maritale', 'Nb enfants',
       'Nb pers à charge', 'Profession du mari', 'Domicile',
       'Niveau socio-économique', 'Tabagisme', 'Neffa', 'Fumées de Tabouna',
       'Antécédents familiaux', 'q', 'Antécédent personnels: chirurgicaux',
       'AT en milieu agricole', 'H travail / jour', 'Mécanisme AT',
       'Ménopause', 'Age ménopause', 'Antécédents gynéco',
       'Ancienneté agricole', 'Produits chimiques utilisés',
       'Troubles gynécologiques', 'Professions antérieures',
       'Catégorie professionnelle', 'Statut', 'Tâches effectuées',
       'Contraintes physiques', 'Engrais utilisés',
       'Produits biologiques utilisés', 'Contraintes thermiques',
       'J travail / Sem', 'Masque pour pesticides', 'Bottes',
       'Niveau scolaire', 'Gants', 'Casquette/Mdhalla', 'Manteau imperméable',
       'Moyen de transport', 'Troubles cardio-respiratoires',
       '

Exploratory Analysis

In [408]:
columns_to_drop = ['N°' ,'Nom', 'Prénom', 'N° du téléphone', 'Date de la visite', 'Date de naissance', 'Age ménopause', 'Antécédents gynéco', 'Niveau scolaire']
df.drop(columns=columns_to_drop, inplace=True)
print(df['Situation maritale'].unique())
# Apply feature engineering to the 'Situation maritale' column
df['Situation maritale'] = df['Situation maritale'].apply(lambda x: 1 if x == 'mariée' else 0)
#treat missing values in the age column
#df['Age'].isna().sum()
# Treat missing values in 'Age' column
df['Age'] = df['Age'].replace({"non spécifié": np.nan, "": np.nan})  # Handle missing values
df['Age'] = pd.to_numeric(df['Age'], errors='coerce')  # Convert to numeric
df['Age'] = df['Age'].fillna(df['Age'].mean()).astype(int)  # Fill missing with mean & convert to int

# Ensure 'Nb enfants' is numeric and handle missing values
df['Nb enfants'] = df['Nb enfants'].replace("non spécifié", np.nan)
df['Nb enfants'] = pd.to_numeric(df['Nb enfants'], errors='coerce').fillna(0).astype(int)
# Treat 'Nb pers à charge' column
df['Nb pers à charge'] = df['Nb pers à charge'].replace("non spécifié", np.nan)  # Handle missing
df['Nb pers à charge'] = pd.to_numeric(df['Nb pers à charge'], errors='coerce')  # Convert to numeric
df['Nb pers à charge'] = df['Nb pers à charge'].fillna(df['Nb pers à charge'].mean()).astype(int)  # Fill missing with mean
# Conditionally update 'Nb pers à charge' only when 'Nb enfants' is greater
df['Nb pers à charge'] = np.where(
    df['Nb enfants'] > df['Nb pers à charge'],  # Condition
    df['Nb enfants'],  # If True, update 'Nb pers à charge' with 'Nb enfants'
    df['Nb pers à charge']  # If False, keep the original value
)
print(df['Nb pers à charge'].unique())
# handle the 'Profession du mari' column
df['Profession du mari'] = df['Profession du mari'].apply(lambda x: 0 if x == 'non spécifié' else 1)
df['Profession du mari'] = df['Profession du mari'].astype(int)
print(df.head(10))
print(df.dtypes)
df = df.iloc[:80]

['célibataire' 'mariée' 'non spécifié' 'divorcée' 'veuve' nan]
[1 4 5 3 2 7 0]
   Age  Situation maritale  Nb enfants  Nb pers à charge  Profession du mari  \
0   60                   0           0                 1                   0   
1   46                   1           1                 1                   1   
2   60                   1           4                 4                   1   
3   48                   1           5                 5                   1   
4   43                   1           3                 3                   1   
5   50                   1           5                 5                   1   
6   34                   0           0                 1                   0   
7   33                   1           3                 3                   1   
8   55                   1           3                 3                   1   
9   52                   1           2                 2                   0   

   Domicile Niveau socio-économique     

In [409]:
# Map categorical values to numerical encoding
socio_economic_mapping = {
    'bon': 2,
    'moyen': 1,
    'bas': 0,
    'non spécifié': np.nan  # Temporarily set to NaN for treatment
}

# Apply mapping
df['Niveau socio-économique'] = df['Niveau socio-économique'].map(socio_economic_mapping)

# Fill missing values with the most frequent (mode) value
df['Niveau socio-économique'].fillna(df['Niveau socio-économique'].mode()[0], inplace=True)

# Convert to integer
df['Niveau socio-économique'] = df['Niveau socio-économique'].astype(int)

# Check results
print(df['Niveau socio-économique'].unique())
print(df[['Niveau socio-économique']].head(10))  # Display first 10 rows

[1 2 0]
   Niveau socio-économique
0                        1
1                        2
2                        1
3                        1
4                        2
5                        1
6                        1
7                        1
8                        0
9                        2


<ipython-input-409-547e941b39c4>:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Niveau socio-économique'].fillna(df['Niveau socio-économique'].mode()[0], inplace=True)


In [410]:
df['Domicile'].unique()
df['Domicile'] = df['Domicile'].fillna(df['Domicile'].mode()[0])  # Fill missing values with mode
df['Domicile'] = df['Domicile'].astype('category')  # Convert to categorical type

print(df['Tabagisme'].unique())
print(df['Neffa'].unique())
# Step 1: Replace missing values with 'non' (non user)
# Step 1: Replace missing values with 'non' (non user)
df['Tabagisme'] = df['Tabagisme'].replace(['non spécifié', np.nan], 'non')
df['Tabagisme'] = df['Tabagisme'].replace(['passif', np.nan], 'oui')
df['Neffa'] = df['Neffa'].replace(['non spécifié', np.nan], 'non')

# Step 2: Convert to binary (1 for 'oui', 0 for 'non')
df['Tabagisme'] = df['Tabagisme'].map({'oui': 1, 'non': 0})
df['Neffa'] = df['Neffa'].map({'oui': 1, 'non': 0})

# Step 3: Create a new column 'TobaccoUse' that combines both columns
# Ensure there are no non-binary values in the columns
df['TobaccoUse'] = ((df['Tabagisme'] == 1) | (df['Neffa'] == 1)).astype(int)

# Drop the old columns if no longer needed
#print(df.dtypes)
df['Fumées de Tabouna'] = df['Fumées de Tabouna'].replace(['non spécifié', np.nan], 'non')
df['Fumées de Tabouna'] = df['Fumées de Tabouna'].map({'oui': 1, 'non': 0})
df.head(20)

['non' 'non spécifié' 'passif' 'oui']
['non' 'non spécifié']


,Age,Situation maritale,Nb enfants,Nb pers à charge,Profession du mari,Domicile,Niveau socio-économique,Tabagisme,Neffa,Fumées de Tabouna,...,Examen buccodentaire,Examen des membres supérieurs,Examen du rachis,Examen des téguments,Examen neurologique,Examen abdominal,Examen visuel,Spirométrie,Interprétation Spiro,TobaccoUse
0,60,0,0,1,0,monastir,1,0,0,1,...,non spécifié,non spécifié,non spécifié,non spécifié,non spécifié,non spécifié,non spécifié,mauvaise exécution,-,0
1,46,1,1,1,1,monastir,2,0,0,1,...,non spécifié,non spécifié,lombalgies,non spécifié,non spécifié,non spécifié,pterygion bilatéral - flou visuel,non faite,-,0
2,60,1,4,4,1,monastir,1,0,0,1,...,non spécifié,douleurs coudes,non spécifié,non spécifié,non spécifié,non spécifié,flou visuel,interprétable,-,0
3,48,1,5,5,1,monastir,1,0,0,1,...,non spécifié,non spécifié,non spécifié,non spécifié,non spécifié,non spécifié,bav,non faite,-,0
4,43,1,3,3,1,monastir,2,0,0,1,...,non spécifié,non spécifié,lombosciatalgies,non spécifié,non spécifié,non spécifié,pterygion droit,interprétable,normale,0
5,50,1,5,5,1,monastir,1,0,0,0,...,non spécifié,non spécifié,ncb droite - sciatalgie droite,non spécifié,non spécifié,non spécifié,non spécifié,non faite,-,0
6,34,0,0,1,0,monastir,1,0,0,0,...,non spécifié,non spécifié,douleur palpation processus épineux,non spécifié,non spécifié,non spécifié,non spécifié,non faite,-,0
7,33,1,3,3,1,monastir,1,0,0,1,...,non spécifié,non spécifié,non spécifié,non spécifié,non spécifié,non spécifié,myopie,interprétable,normale,0
8,55,1,3,3,1,monastir,0,0,0,1,...,non spécifié,non spécifié,non spécifié,non spécifié,non spécifié,sensibilité épigastrique,non spécifié,non faite,-,0
9,52,1,2,2,0,monastir,2,0,0,1,...,non spécifié,non spécifié,non spécifié,non spécifié,non spécifié,non spécifié,non spécifié,interprétable,normale,0


In [411]:
print(df['Moyen de transport'].unique())
modes = ['a pieds', 'charette', 'charrette', 'non spécifié', 'bus / transport public',
         'a pieds, camion non protégé', 'a pieds, bus / transport public', 'voiture',
         'a pieds, charette', 'a pieds, camion non protégé, charette', 'a pieds, voiture',
         'camion non protégé', 'bus / transport public, camion non protégé']

# Create binary columns for each mode of transport
for mode in modes:
    mode = mode.strip()  # remove leading/trailing whitespaces
    df[mode] = df['Moyen de transport'].apply(lambda x: 1 if mode in str(x) else 0)

# Check the updated dataframe
print(df.head())

df['Poids'] = pd.to_numeric(df['Poids'], errors='coerce')
df['Taille'] = pd.to_numeric(df['Taille'], errors='coerce')

df['BMI'] = df['Poids'] / (df['Taille'] / 100) ** 2
df.head(10)
df['Manteau imperméable'].unique()

['a pieds' 'charette' 'charrette' 'non spécifié' 'bus / transport public'
 'a pieds, camion non protégé' 'a pieds, bus / transport public' 'voiture'
 'a pieds, charette' 'a pieds, camion non protégé, charette'
 'a pieds, voiture' 'camion non protégé'
 'bus / transport public, camion non protégé']
   Age  Situation maritale  Nb enfants  Nb pers à charge  Profession du mari  \
0   60                   0           0                 1                   0   
1   46                   1           1                 1                   1   
2   60                   1           4                 4                   1   
3   48                   1           5                 5                   1   
4   43                   1           3                 3                   1   

   Domicile  Niveau socio-économique  Tabagisme  Neffa  Fumées de Tabouna  \
0  monastir                        1          0      0                  1   
1  monastir                        2          0      0             

array(['jamais', 'souvent', 'parfois', 'toujours'], dtype=object)

Setting the target variables

In [412]:
# Define a function to handle the replacement for each column
def fill_troubles_column(df, column_name):
    df[column_name] = df[column_name].apply(lambda x: 0 if x == 'non spécifié' else 1)

# Apply the function to each of the four columns
fill_troubles_column(df, 'Troubles cardio-respiratoires')
fill_troubles_column(df, 'Troubles neurologiques')
fill_troubles_column(df, 'Troubles cutanés/phanères')
fill_troubles_column(df, 'Troubles cognitifs')

print(df["Troubles cutanés/phanères"])

# Initialize the encoder
encoder = LabelEncoder()

# Encode the 'Manteau imperméable' column
df['Manteau imperméable'] = encoder.fit_transform(df['Manteau imperméable']).astype(int)
df['Gants'] = encoder.fit_transform(df['Gants']).astype(int)
df['Casquette/Mdhalla'] = encoder.fit_transform(df['Manteau imperméable']).astype(int)
df['Bottes'] = encoder.fit_transform(df['Bottes']).astype(int)
df['Masque pour pesticides'] = encoder.fit_transform(df['Masque pour pesticides']).astype(int)
# Check the updated values
print(df['Manteau imperméable'].unique())
print(df['Gants'].unique())
print(df['Casquette/Mdhalla'].unique())
print(df.dtypes)
# One-hot encoding for the 'Domicile' column
df = pd.get_dummies(df, columns=['Domicile'], drop_first=True)

# Check the updated dataframe
print(df.head())
df['J travail / Sem'] = df['J travail / Sem'].replace(['non spécifié'], np.nan)
df['H travail / jour'] = df['H travail / jour'].replace(['non spécifié'], np.nan)
df['J travail / Sem'] = pd.to_numeric(df['J travail / Sem'], errors='coerce')  # Convert to numeric
df['J travail / Sem'] = df['J travail / Sem'].fillna(df['J travail / Sem'].mean()).astype(int)  # Fill missing with mean & convert to int
df['H travail / jour'] = pd.to_numeric(df['H travail / jour'], errors='coerce')  # Convert to numeric
df['H travail / jour'] = df['H travail / jour'].fillna(df['H travail / jour'].mean()).astype(int)  # Fill missing with mean & convert to int
df['H_travail_sem'] = df['H travail / jour'] * df['J travail / Sem']
print(df['H_travail_sem'])

0     0
1     0
2     0
3     0
4     0
     ..
75    0
76    0
77    0
78    0
79    1
Name: Troubles cutanés/phanères, Length: 80, dtype: int64
[0 2 1 3]
[2 0 1 3]
[0 2 1 3]
Age                                             int64
Situation maritale                              int64
Nb enfants                                      int64
Nb pers à charge                                int64
Profession du mari                              int64
                                               ...   
a pieds, camion non protégé, charette           int64
a pieds, voiture                                int64
camion non protégé                              int64
bus / transport public, camion non protégé      int64
BMI                                           float64
Length: 71, dtype: object
   Age  Situation maritale  Nb enfants  Nb pers à charge  Profession du mari  \
0   60                   0           0                 1                   0   
1   46                   1           1      

In [413]:
df['nombre_des_taches'] = df['Tâches effectuées'].apply(lambda x: len(str(x).split(',')) if pd.notna(x) else 0)
df['nombre_des_taches']
df.columns

Index(['Age', 'Situation maritale', 'Nb enfants', 'Nb pers à charge',
       'Profession du mari', 'Niveau socio-économique', 'Tabagisme', 'Neffa',
       'Fumées de Tabouna', 'Antécédents familiaux', 'q',
       'Antécédent personnels: chirurgicaux', 'AT en milieu agricole',
       'H travail / jour', 'Mécanisme AT', 'Ménopause', 'Ancienneté agricole',
       'Produits chimiques utilisés', 'Troubles gynécologiques',
       'Professions antérieures', 'Catégorie professionnelle', 'Statut',
       'Tâches effectuées', 'Contraintes physiques', 'Engrais utilisés',
       'Produits biologiques utilisés', 'Contraintes thermiques',
       'J travail / Sem', 'Masque pour pesticides', 'Bottes', 'Gants',
       'Casquette/Mdhalla', 'Manteau imperméable', 'Moyen de transport',
       'Troubles cardio-respiratoires', 'Troubles cognitifs',
       'Troubles neurologiques', 'Troubles cutanés/phanères',
       'Autres plaintes:', 'Poids', 'Taille', 'TAS', 'TAD', 'GAD',
       'Examen cardiovasculaire 

Choosing the columns to keep

In [414]:
columns_to_keep = ['Age', 'Situation maritale', 'Nb pers à charge', 'Profession du mari', 'Niveau socio-économique', 'Fumées de Tabouna', 'nombre_des_taches', 'Troubles cardio-respiratoires', 'Troubles cognitifs',
       'Troubles neurologiques', 'Troubles cutanés/phanères', 'bus / transport public',
       'a pieds, camion non protégé', 'a pieds, bus / transport public',
       'voiture', 'a pieds, charette', 'a pieds, camion non protégé, charette',
       'a pieds, voiture', 'camion non protégé',
       'bus / transport public, camion non protégé', 'BMI',
       'Domicile_monastir', 'Domicile_sfax', 'H_travail_sem',
       'nombre_des_taches', 'Masque pour pesticides', 'Bottes', 'Gants', 'Casquette/Mdhalla', 'BMI', 'TAS']
cleaned_df = df[columns_to_keep]
df.head()
cleaned_df.to_csv('cleaned_data.csv', index=False)
